In [1]:
#%run ./methods.ipynb

In [2]:
#df = pd.read_pickle('./data/fitbitflexwirelessactivitysleepwristbandblack.pkl')

In [3]:
#%run ./tfidf.ipynb

In [4]:
# add and remove stopwords
stopwords = ['iphone', 'fitbit', 'alta', 'fit bit', 'hr', 'sony', 'mdr', 'mdr7506', 'product', 'device', 'thing', 'love','daughter','son','husband','wife','mother','father','star','pair','thing','product','mom','dad','day','week','month','year','minute','second']
no_stopwords = ['no', 'not']
stopword_list = nltk.corpus.stopwords.words('english')
for stopword in stopwords:
    stopword_list.append(stopword)
for no_stopword in no_stopwords:
    stopword_list.remove(no_stopword)

In [5]:
# extract noun chunks from sentences and clear
review_noun_phrases = []
for review in df.preprocessed:
    
    # split reviews into sentences and extract noun chunks
    review_noun_chunks = [
        nlp(review).noun_chunks for review in sent_tokenize(review)
    ]

    # convert noun chunks to string and remove single word noun phrases
    review_noun_chunks_corpus = []
    for review_noun_chunk in review_noun_chunks:
        tmp_noun_chunks = []
        for chunk in review_noun_chunk:
            if len(nltk.word_tokenize(chunk.text)) > 1:
                tmp_noun_chunks.append(chunk.text)
        if tmp_noun_chunks:
            review_noun_chunks_corpus = review_noun_chunks_corpus + tmp_noun_chunks

    # preprocess noun phrases
    preprocessed_noun_chunks = [
        preprocess(
            noun_chunk,
            isolate_special_characters=True,
            remove_special_characters=True,
            remove_extra_whitespace=True,
            remove_stopwords=True,
            remove_numbers=True) for noun_chunk in review_noun_chunks_corpus
    ]

    # remove single word noun phrases
    sentence_noun_phrases = []
    for sentence in preprocessed_noun_chunks:
        if len(nltk.word_tokenize(sentence)) > 1:
            sentence_noun_phrases.append(sentence)

    review_noun_phrases.append(sentence_noun_phrases)

In [6]:
review_noun_phrases_nouns= []
for review in review_noun_phrases:
    tmp_noun_phrases_entities = []
    for noun_chunk in review:
        tmp_noun_phrases_entities.append(extract_entity(noun_chunk))
    review_noun_phrases_nouns.append(tmp_noun_phrases_entities)

In [7]:
merged = list(itertools.chain(*review_noun_phrases_nouns))
merged2 = list(itertools.chain(*review_noun_phrases))

In [8]:
counter = Counter(merged2)

In [9]:
score = []
for i,review in enumerate(review_noun_phrases_nouns):
    tmp_topic = []
    for j,ph in enumerate(review):
        try:
            tmp_topic.append((sum([d[preprocess(word, lemmatize=True)]for word in ph.split()])/len(ph.split()))*len(review_noun_phrases[i][j].split()))
        except:
            tmp_topic.append(None)
    score.append(tmp_topic)

In [10]:
df["noun_phrases"] = review_noun_phrases
df["noun_phrase_scores"] = score
df["noun_phrase_entities"] = review_noun_phrases_nouns
#df.to_pickle('./data/fitbitflexwirelessactivitysleepwristbandblack.pkl')

In [45]:
#flatten1 = list(itertools.chain(*review_noun_phrases))
#flatten2 = list(itertools.chain(*score))

In [46]:
#d = {'np': flatten1, 'score': flatten2}
#lel = pd.DataFrame(data=d)

In [47]:
#lel = lel.drop_duplicates()

In [48]:
#lel.sort_values(by=['score'], ascending=False)